In [10]:
import numpy as np

from sklearn.metrics import accuracy_score

import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as F

In [11]:
#Load data function

def read_bci_data():
    S4b_test = np.load('S4b_test.npz')
    X11b_test = np.load('X11b_test.npz')

    test_data = np.concatenate((S4b_test['signal'], X11b_test['signal']), axis=0)
    test_label = np.concatenate((S4b_test['label'], X11b_test['label']), axis=0)
    test_label = test_label -1

    test_data = np.transpose(np.expand_dims(test_data, axis=1), (0, 1, 3, 2))

    mask = np.where(np.isnan(test_data))
    test_data[mask] = np.nanmean(test_data)

    print(test_data.shape, test_label.shape)

    return test_data, test_label

<h3>EEGNet and DeepConvNet function</h3>

In [12]:
class EEGNet(nn.Module):
    def __init__(self,activation_name):
        super(EEGNet, self).__init__()
        
        # Layer 1
        self.conv1 = nn.Conv2d(1, 16, (1, 51), padding = (0,25),bias=False)
        self.batchnorm1 = nn.BatchNorm2d(16, eps=1e-5)
        
        # Layer 2
        self.conv2 = nn.Conv2d(16, 32, (2, 1),groups=16,bias=False)
        self.batchnorm2 = nn.BatchNorm2d(32, eps=1e-5)
        if activation_name=='relu':
            self.act1=nn.ReLU()
        elif activation_name=='leakyrelu':
            self.act1=nn.LeakyReLU()
        elif activation_name=='elu':
            self.act1=nn.ELU()
        self.pooling2 = nn.AvgPool2d((1, 4))
        self.drop1 = nn.Dropout(p=0.5)
        
        # Layer 3
        self.conv3 = nn.Conv2d(32, 32, (1, 15),padding = (0,7),bias=False)
        self.batchnorm3 = nn.BatchNorm2d(32, eps=1e-5)
        if activation_name=='relu':
            self.act2=nn.ReLU()
        elif activation_name=='leakyrelu':
            self.act2=nn.LeakyReLU()
        elif activation_name=='elu':
            self.act2=nn.ELU()
        self.pooling3 = nn.AvgPool2d((1, 8))
        self.drop2 = nn.Dropout(p=0.5)
        
        self.fc1 = nn.Linear(736, 2,bias=True)
        

    def forward(self, x):
        # Layer 1
        x = self.conv1(x)
        x = self.batchnorm1(x)
        
        # Layer 2
        x = self.conv2(x)
        x = self.batchnorm2(x)
        x = self.act1(x)
        x = self.pooling2(x)
        x = self.drop1(x)
        
        # Layer 3
        x = self.conv3(x)
        x = self.batchnorm3(x)
        x = self.act2(x)
        x = self.pooling3(x)
        x = self.drop2(x)
        
        # FC Layer
        x = x.view(-1, 736)
        x = F.sigmoid(self.fc1(x))
        
        return x


class DeepConvNet(nn.Module):
    def __init__(self,activation_name):
        super(DeepConvNet, self).__init__()
        
        # Layer 1
        self.conv1 = nn.Conv2d(1, 25, (1, 5))
        self.conv2 = nn.Conv2d(25, 25, (2, 1))
        self.batchnorm1 = nn.BatchNorm2d(25, eps=1e-5)
        if activation_name=='relu':
            self.act1=nn.ReLU()
        elif activation_name=='leakyrelu':
            self.act1=nn.LeakyReLU()
        elif activation_name=='elu':
            self.act1=nn.ELU()
        self.pooling1 = nn.MaxPool2d((1, 2))
        self.drop1 = nn.Dropout(p=0.25)
        
        
        # Layer 2
        self.conv3 = nn.Conv2d(25, 50, (1, 5))
        self.batchnorm2 = nn.BatchNorm2d(50, eps=1e-5)
        if activation_name=='relu':
            self.act2=nn.ReLU()
        elif activation_name=='leakyrelu':
            self.act2=nn.LeakyReLU()
        elif activation_name=='elu':
            self.act2=nn.ELU()
        self.pooling2 = nn.MaxPool2d((1, 2))
        self.drop2 = nn.Dropout(p=0.25)
        
        # Layer 3
        self.conv4 = nn.Conv2d(50, 100, (1, 5))
        self.batchnorm3 = nn.BatchNorm2d(100, eps=1e-5)
        if activation_name=='relu':
            self.act3=nn.ReLU()
        elif activation_name=='leakyrelu':
            self.act3=nn.LeakyReLU()
        elif activation_name=='elu':
            self.act3=nn.ELU()
        self.pooling3 = nn.MaxPool2d((1, 2))
        self.drop3 = nn.Dropout(p=0.25)
        
        # Layer 4
        self.conv5 = nn.Conv2d(100, 200, (1, 5))
        self.batchnorm4 = nn.BatchNorm2d(200, eps=1e-5)
        if activation_name=='relu':
            self.act4=nn.ReLU()
        elif activation_name=='leakyrelu':
            self.act4=nn.LeakyReLU()
        elif activation_name=='elu':
            self.act4=nn.ELU()
        self.pooling4 = nn.MaxPool2d((1, 2))
        self.drop4 = nn.Dropout(p=0.25)
        
        self.fc1 = nn.Linear(8600, 2,bias=True)
        

    def forward(self, x):
        # Layer 1
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.batchnorm1(x)
        x = self.act1(x)
        x = self.pooling1(x)
        x = self.drop1(x)
        
        # Layer 2
        x = self.conv3(x)
        x = self.batchnorm2(x)
        x = self.act2(x)
        x = self.pooling2(x)
        x = self.drop2(x)
        
        # Layer 3
        x = self.conv4(x)
        x = self.batchnorm3(x)
        x = self.act3(x)
        x = self.pooling3(x)
        x = self.drop3(x)

        # Layer 4
        x = self.conv5(x)
        x = self.batchnorm4(x)
        x = self.act4(x)
        x = self.pooling4(x)
        x = self.drop4(x)
        
        # FC Layer
        x = x.view(-1, 8600)
        x = F.softmax(self.fc1(x))
        return x

<h3>Evaluate function</h3>

In [13]:
def evaluate(model, X, Y):
    results = []
    batch_size = 72
    
    predicted = []
    
    for i in range(15):
        s = i*batch_size
        e = i*batch_size+batch_size
        
        inputs = torch.from_numpy(X[s:e])
        inputs = Variable(inputs.to(device='cuda:0', dtype=torch.float))
        pred = model(inputs)
        
        predicted.append(pred.data.cpu().numpy())
        
    inputs = Variable(torch.from_numpy(X).to(device='cuda:0', dtype=torch.float))
    predicted = model(inputs)
    
    predicted = predicted.data.cpu().numpy()
            
    acc=accuracy_score(Y, np.argmax(predicted,axis=-1))
            
    return acc

<h3>Create model</h3>

In [14]:
eeg_elu_net = EEGNet("elu").cuda(0)
eeg_relu_net = EEGNet("relu").cuda(0)
eeg_leakyrelu_net = EEGNet("leakyrelu").cuda(0)
deep_conv_elu_net = DeepConvNet("elu").cuda(0)
deep_conv_relu_net = DeepConvNet("relu").cuda(0)
deep_conv_leakyrelu_net = DeepConvNet("leakyrelu").cuda(0)

<h3>Model structure</h3>

In [15]:
from torchsummary import summary

summary(eeg_elu_net, (1, 2, 750))
summary(eeg_relu_net, (1, 2, 750))
summary(eeg_leakyrelu_net, (1, 2, 750))
summary(deep_conv_elu_net, (1, 2, 750))
summary(deep_conv_relu_net, (1, 2, 750))
summary(deep_conv_leakyrelu_net, (1, 2, 750))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 2, 750]             816
       BatchNorm2d-2           [-1, 16, 2, 750]              32
            Conv2d-3           [-1, 32, 1, 750]              64
       BatchNorm2d-4           [-1, 32, 1, 750]              64
               ELU-5           [-1, 32, 1, 750]               0
         AvgPool2d-6           [-1, 32, 1, 187]               0
           Dropout-7           [-1, 32, 1, 187]               0
            Conv2d-8           [-1, 32, 1, 187]          15,360
       BatchNorm2d-9           [-1, 32, 1, 187]              64
              ELU-10           [-1, 32, 1, 187]               0
        AvgPool2d-11            [-1, 32, 1, 23]               0
          Dropout-12            [-1, 32, 1, 23]               0
           Linear-13                    [-1, 2]           1,474
Total params: 17,874
Trainable params: 

D:\anaconda3\lib\site-packages\ipykernel_launcher.py:151: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


<h3>Load pre-training model</h3>

In [16]:
eeg_elu_model = torch.load('./model/eegnet_elu_8333.pt')
eeg_relu_model = torch.load('./model/eegnet_relu_8638.pt')
egg_leaky_model = torch.load('./model/eegnet_laekyrelu_8611.pt')
deep_conv_elu_model = torch.load('./model/deepconvnet_elu_8703.pt')
deep_conv_relu_model = torch.load('./model/deepconvnet_relu_875.pt')
deep_conv_leakyrelu_model = torch.load('./model/deepconvnet_laekyrelu_8722.pt')
eeg_elu_model.eval()
eeg_relu_model.eval()
egg_leaky_model.eval()
deep_conv_elu_model.eval()
deep_conv_relu_model.eval()
deep_conv_leakyrelu_model.eval()

DeepConvNet(
  (conv1): Conv2d(1, 25, kernel_size=(1, 5), stride=(1, 1))
  (conv2): Conv2d(25, 25, kernel_size=(2, 1), stride=(1, 1))
  (batchnorm1): BatchNorm2d(25, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (act1): LeakyReLU(negative_slope=0.01)
  (pooling1): MaxPool2d(kernel_size=(1, 2), stride=(1, 2), padding=0, dilation=1, ceil_mode=False)
  (drop1): Dropout(p=0.25, inplace=False)
  (conv3): Conv2d(25, 50, kernel_size=(1, 5), stride=(1, 1))
  (batchnorm2): BatchNorm2d(50, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (act2): LeakyReLU(negative_slope=0.01)
  (pooling2): MaxPool2d(kernel_size=(1, 2), stride=(1, 2), padding=0, dilation=1, ceil_mode=False)
  (drop2): Dropout(p=0.25, inplace=False)
  (conv4): Conv2d(50, 100, kernel_size=(1, 5), stride=(1, 1))
  (batchnorm3): BatchNorm2d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (act3): LeakyReLU(negative_slope=0.01)
  (pooling3): MaxPool2d(kernel_size=(1, 2), strid

<h3>Create data</h3>

In [17]:
test_data, test_label=read_bci_data()

(1080, 1, 2, 750) (1080,)


<h3>Testing</h3>

In [18]:
test_acc = evaluate(eeg_elu_model, test_data, test_label)
print("Accuracy of EEGNet on test set(ELU) : ",np.round(test_acc*100,2),"%")
test_acc = evaluate(eeg_relu_model, test_data, test_label)
print("Accuracy of EEGNet on test set(ReLU) : ",np.round(test_acc*100,2),"%")
test_acc = evaluate(egg_leaky_model, test_data, test_label)
print("Accuracy of EEGNet on test set(LeakyReLU) : ",np.round(test_acc*100,2),"%")
test_acc = evaluate(deep_conv_elu_model, test_data, test_label)
print("Accuracy of DeepConvNet on test set(ELU) : ",np.round(test_acc*100,2),"%")
test_acc = evaluate(deep_conv_relu_model, test_data, test_label)
print("Accuracy of DeepConvNet on test set(ReLU) : ",np.round(test_acc*100,2),"%")
test_acc = evaluate(deep_conv_leakyrelu_model, test_data, test_label)
print("Accuracy of DeepConvNet on test set(LeakyReLU) : ",np.round(test_acc*100,2),"%")

Accuracy of EEGNet on test set(ELU) :  83.8 %
Accuracy of EEGNet on test set(ReLU) :  87.22 %
Accuracy of EEGNet on test set(LeakyReLU) :  88.61 %


D:\anaconda3\lib\site-packages\ipykernel_launcher.py:151: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Accuracy of DeepConvNet on test set(ELU) :  86.94 %
Accuracy of DeepConvNet on test set(ReLU) :  87.87 %
Accuracy of DeepConvNet on test set(LeakyReLU) :  87.87 %
